In [1]:
from stochastic_volatility_models.src.models.rough_bergomi.rough_bergomi import RoughBergomi
from stochastic_volatility_models.src.models.heston.heston import Heston
from stochastic_volatility_models.src.core.evaluation_functions import surface_atm_skew, surface_evaluation
from stochastic_volatility_models.src.utils.options.skew import atm_skew

from notebook_utils import Notebook

2024-08-09 13:51:47.327 | INFO     | stochastic_volatility_models.config:initialise:30 - Loaded module `stochastic_volatility_models` from project directory path `/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/Project/modules`


In [2]:
nb_e = Notebook(
	model=None,
)
nb_h = Notebook(
	model=Heston(
		parameters={
			"initial_variance": 0.10006713187966065,
			"long_term_variance": 0.03159120049914011,
			"volatility_of_volatility": 3.4949236489765254,
			"mean_reversion_rate": 6.106005002941104,
			"wiener_correlation": -0.7977506958223608,
		},
	),
)
nb_rb = Notebook(
	model=RoughBergomi(
		parameters={
			"hurst_index": 0.600784070323841,
			"wiener_correlation": -0.9283329233829251,
			"volatility_of_volatility": 6.196125566995583,
		},
	),
)

In [3]:
volatility_surface = nb_rb.spx_vs
surfaces = [
	volatility_surface.surface_quantities(
		time=nb_rb.time,
		quantity_method="empirical_pricing_implied_volatility",
		price_types=["Mid"],
		pricing_model=nb_e.pricing_model,
	)[0],
	volatility_surface.surface_quantities(
		time=nb_rb.time,
		quantity_method="model_pricing_implied_volatility",
		price_types=["Mid"],
		model=nb_rb.model,
		pricing_model=nb_rb.pricing_model,
	)[0],
]

atm_skews = [
	atm_skew(
		surface=surface,
		volatility_surface=volatility_surface,
		time=nb_rb.time,
	).values
	for surface in surfaces
]

In [4]:
atm_skews

[array([[1.98547473],
        [1.97002743],
        [1.6173868 ],
        [1.41155758],
        [1.29418723],
        [1.17658411],
        [1.19820093],
        [1.18522719],
        [1.11691653],
        [1.04858406],
        [0.97011532],
        [0.92689096],
        [0.82860037],
        [0.79450431],
        [0.74192256],
        [0.70467937],
        [0.65944621],
        [0.62420632],
        [0.57344434]]),
 array([[1.7927948 ],
        [1.63922197],
        [1.30990256],
        [1.46712251],
        [1.68639569],
        [1.24162735],
        [1.24722422],
        [1.21168172],
        [1.13931828],
        [0.85553458],
        [1.06097024],
        [0.96333149],
        [1.08923454],
        [0.92924415],
        [0.7454452 ],
        [0.78459982],
        [0.77599776],
        [0.72309421],
        [0.64307117]])]

In [9]:
surface_atm_skew(
	volatility_surface=nb_rb.spx_vs,
	time=nb_rb.time,
	model=nb_rb.model,  # type: ignore
	empirical_pricing_model=nb_rb.empirical_pricing_model,
	model_pricing_model=nb_rb.pricing_model,
) * 0.01

0.008686053463315194

In [6]:
surface_evaluation(
	volatility_surface=nb_rb.spx_vs,
	time=nb_rb.time,
	model=nb_rb.model,  # type: ignore
	empirical_pricing_model=nb_rb.empirical_pricing_model,
	model_pricing_model=nb_rb.pricing_model,
)

0.004689040317522883

In [14]:
surfaces = [
	volatility_surface.surface_quantities(
		time=nb_rb.time,
		quantity_method="model_pricing_implied_volatility",
		price_types=["Mid"],
		model=nb_rb.model,
		pricing_model=nb_rb.pricing_model,
		call=True,
	)[0],
	volatility_surface.surface_quantities(
		time=nb_rb.time,
		quantity_method="model_pricing_implied_volatility",
		price_types=["Mid"],
		model=nb_rb.model,
		pricing_model=nb_rb.pricing_model,
		call=False,
	)[0],
]
diff = surfaces[0] - surfaces[1]
diff

Symbol
Strike Expiry              
2200   2022-03-09  0.414393
       2022-03-11  0.443971
       2022-03-18  0.259215
       2022-03-23  0.099381
       2022-03-25  0.010705
...                     ...
5400   2022-06-17  0.151833
       2022-06-30  0.143241
       2022-07-15  0.135097
       2022-07-29  0.128619
       2022-08-31  0.117029

[1577 rows x 1 columns]

: 